In [1]:
#Importing library

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
#import xgboost as xgb

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr
from sklearn.datasets import make_regression
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
df = pd.read_csv("dataset/train_processed.csv")

In [3]:
df_date=df['Date']
df_no_date=df.drop(['Date'], axis=1)
df_norm= (df_no_date - df_no_date.min()) / (df_no_date.max() - df_no_date.min())
df=df_norm
df['Date']=df_date

In [4]:
df_val3=df.loc[(df['Date']>='2018-01-01')]
df_val3_rest=df.loc[(df['Date']<'2018-01-01')]
df_val2=df_val3_rest.loc[(df_val3_rest['Date']>='2017-11-01')]
df_val2_rest=df.loc[(df['Date']<'2017-11-01')]
df_val1=df_val2_rest.loc[(df_val2_rest['Date']>='2017-09-01')]
df_train=df.loc[(df['Date']<'2017-09-01')]

In [5]:
df_train_clear=df_train.drop(['Date'], axis=1)
df_val1_clear=df_val1.drop(['Date'], axis=1)
df_val2_clear=df_val2.drop(['Date'], axis=1)
df_val3_clear=df_val3.drop(['Date'], axis=1)

In [6]:
y=df_train_clear['NumberOfSales']
y_val1=df_val1_clear['NumberOfSales']
y_val2=df_val2_clear['NumberOfSales']
y_val3=df_val3_clear['NumberOfSales']

In [7]:
X=df_train_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers', 'Unnamed: 0'], axis=1)
X_val1=df_val1_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers', 'Unnamed: 0'], axis=1)
X_val2=df_val2_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers', 'Unnamed: 0'], axis=1)
X_val3=df_val3_clear.drop(['StoreID','NumberOfSales','NumberOfCustomers', 'Unnamed: 0'], axis=1)

In [8]:
columns=['eta','max_depth', 'colsample_bytree', 'n_estimators','min_child_weight','gamma', 'subsample', mse', 'mae']
index=range(1,1125000)
hp=pd.DataFrame(index=index,columns=columns)

index = 0
for eta in [0.1, 0.2, 0.3]:
    for max_depth in [3, 6, 10]:
        for colsample_bytree in [0.6,0.8]:
            for n_estimators in range(50, 201, 50):
                xclas = XGBRegressor(subsample=0.6, eta=eta, max_depth=max_depth, 
                                     colsample_bytree=colsample_bytree, nthread=4,
                                     n_estimators=n_estimators)  
                xclas.fit(X, y)   
                y_pred = xclas.predict(X_val1) 
                mse_val1=mean_squared_error(y_val1,y_pred)
                mae_val1=mean_absolute_error(y_val1,y_pred)
                index +=1
                hp.at[index,'eta']=eta
                hp.at[index,'max_depth']=max_depth
                hp.at[index,'colsample_bytree']=colsample_bytree
                hp.at[index, 'n_estimators']=n_estimators
                hp.at[index, 'mae']=mae_val1
                hp.at[index,'mse']=sqrt(mse_val1)
                print(eta, max_depth, colsample_bytree, n_estimators, sqrt(mse_val1), mae_val1)

In [40]:
index = 0

n_estimators=75
eta=0.3
max_depth=9
min_child_weight=5
gamma=0
subsample=0.6
colsample_bytree=0.55

xclas = XGBRegressor(subsample=subsample, eta=eta, max_depth=max_depth, 
                     colsample_bytree=colsample_bytree, nthread=4,
                     min_child_weight=min_child_weight, gamma=gamma,
                     n_estimators=n_estimators)  
xclas.fit(X, y)   
y_pred = xclas.predict(X_val1) 
mse_val1=mean_squared_error(y_val1,y_pred)
mae_val1=mean_absolute_error(y_val1,y_pred)
index +=1
hp.at[index,'eta']=eta
hp.at[index,'max_depth']=max_depth
hp.at[index,'colsample_bytree']=colsample_bytree
hp.at[index, 'n_estimators']=n_estimators
hp.at[index, 'gamma']=gamma
hp.at[index, 'subsample']=subsample
hp.at[index, 'min_child_weight']=min_child_weight
hp.at[index, 'mae']=mae_val1
hp.at[index,'mse']=sqrt(mse_val1)
print(eta, max_depth, colsample_bytree, n_estimators, min_child_weight, gamma, sqrt(mse_val1), mae_val1)

0.3 9 0.55 75 5 0 0.029446591057216216 0.0208359766567


In [41]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(hp.loc[(hp['mse'] < 0.0295)])

   eta max_depth colsample_bytree n_estimators        mse       mae  gamma  \
1  0.3         9             0.55           75  0.0294466  0.020836    0.0   
6  0.3         9             0.55           75  0.0294466  0.020836    0.0   

   subsample  min_child_weight  
1        0.6               5.0  
6        0.6               5.0  
